<a href="https://colab.research.google.com/github/barrosyan/AstroML/blob/main/Studyohysicsnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Particle properties
N = 100  # Number of particles
m = np.ones(N) * 1e6  # Masses of particles in solar masses
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Integration parameters
dt = 0.01  # Time step size

# Gravitational force calculation
def gravity_force(t, y):
    positions = y[:2*N].reshape((N, 2))
    velocities = y[2*N:].reshape((N, 2))
    forces = np.zeros_like(positions)

    for i in range(N):
        for j in range(N):
            if i != j:
                r_diff = positions[j] - positions[i]
                distance = np.linalg.norm(r_diff)
                forces[i] += m[i] * m[j] * r_diff / distance**3

    return np.concatenate((velocities.flatten(), forces.flatten()))

# Solve the equations of motion using scipy.integrate.solve_ivp
y0 = np.concatenate((r.flatten(), v.flatten()))
solution = solve_ivp(gravity_force, (0, 10), y0, method='RK45', t_eval=np.arange(0, 10, dt))

# Extract positions and velocities from the solution
r_final = solution.y[:2*N, -1].reshape((N, 2))
v_final = solution.y[2*N:, -1].reshape((N, 2))

# Print the final positions and velocities
print("Final Positions:", r_final)
print("Final Velocities:", v_final)

# Plot the final positions
plt.scatter(r_final[:, 0], r_final[:, 1])
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Final Particle Positions')
plt.show()

KeyboardInterrupt: ignored

In [ ]:
import numpy as np

# Particle properties
N = 100  # Number of particles
m = np.ones(N)  # Masses of particles
r = np.random.uniform(low=-1.0, high=1.0, size=(N, 3))  # Positions of particles (randomly initialized)
rho = np.ones(N)  # Density of particles (initially set to 1)

# Calculate hydrodynamic forces
F_hydro = np.zeros((N, 3))  # Array to store hydrodynamic forces

# Constants
kernel_radius = 0.5  # Radius of the kernel function

# Kernel function
def W(q, h):
    # Smoothing kernel
    q_norm = np.linalg.norm(q)
    coeff = 315 / (64 * np.pi * h**9)

    if 0 <= q_norm < h:
        return coeff * (h**2 - q_norm**2)**3
    else:
        return 0

# Calculate hydrodynamic forces
for i in range(N):
    for j in range(N):
        if i != j:
            r_diff = r[j] - r[i]
            distance = np.linalg.norm(r_diff)

            if distance < kernel_radius:
                kernel = W(r_diff, kernel_radius)
                pressure_force = -m[j] * ((rho[i] + rho[j]) / 2) * (1 / rho[i]**2) * (pressure[i] + pressure[j])
                viscous_force = -2 * mu * ((v[j] - v[i]) / rho[i]) * (1 / rho[i]) * kernel

                F_hydro[i] += (pressure_force + viscous_force) * r_diff / distance

# Print the hydrodynamic forces
print(F_hydro)

In [ ]:

import numpy as np

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
radius = 0.1  # Radius for neighboring particle search

# Grid parameters
grid_size = 10  # Size of the grid
cell_size = radius  # Size of each grid cell

# Initialize the grid
num_cells = int(np.ceil(grid_size / cell_size))
grid = [[] for _ in range(num_cells * num_cells)]

# Assign particles to the grid cells
for i in range(N):
    cell_x = int(r[i, 0] / cell_size)
    cell_y = int(r[i, 1] / cell_size)
    cell_index = cell_x + cell_y * num_cells
    grid[cell_index].append(i)

# Find neighboring particles for each particle
neighboring_particles = [[] for _ in range(N)]

for i in range(N):
    cell_x = int(r[i, 0] / cell_size)
    cell_y = int(r[i, 1] / cell_size)
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            neighbor_x = cell_x + dx
            neighbor_y = cell_y + dy
            if 0 <= neighbor_x < num_cells and 0 <= neighbor_y < num_cells:
                neighbor_cell_index = neighbor_x + neighbor_y * num_cells
                for j in grid[neighbor_cell_index]:
                    if np.linalg.norm(r[i] - r[j]) < radius:
                        neighboring_particles[i].append(j)

# Print the neighboring particles
for i, neighbors in enumerate(neighboring_particles):
    print(f"Particle {i} has neighbors: {neighbors}")

In [ ]:

import numpy as np

# Particle properties
N = 100  # Number of particles
m = np.ones(N)  # Masses of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Integration parameters
dt = 0.01  # Time step size
G = 1.0  # Gravitational constant

# Compute forces and update velocities and positions
for _ in range(100):
    # Compute gravitational forces
    F_gravity = np.zeros((N, 2))
    for i in range(N):
        for j in range(N):
            if i != j:
                r_diff = r[j] - r[i]
                distance = np.linalg.norm(r_diff)
                F_gravity[i] += G * m[i] * m[j] * r_diff / distance**3

    # Update velocities (Leapfrog half-step)
    v += 0.5 * F_gravity * dt

    # Update positions
    r += v * dt

    # Compute forces again
    F_gravity_new = np.zeros((N, 2))
    for i in range(N):
        for j in range(N):
            if i != j:
                r_diff = r[j] - r[i]
                distance = np.linalg.norm(r_diff)
                F_gravity_new[i] += G * m[i] * m[j] * r_diff / distance**3

    # Update velocities again (Leapfrog full-step)
    v += 0.5 * F_gravity_new * dt

# Print the final positions and velocities
print("Final Positions:", r)
print("Final Velocities:", v)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Particle interpolation using spline interpolation
spline = UnivariateSpline(r[:, 0], r[:, 1])
x_interpolated = np.linspace(0, 1, 100)
y_interpolated = spline(x_interpolated)

# Plot the original and interpolated particle positions
plt.scatter(r[:, 0], r[:, 1], label='Original')
plt.plot(x_interpolated, y_interpolated, color='red', label='Interpolated')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Particle Interpolation')
plt.legend()
plt.show()

In [ ]:

import numpy as np

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Simulation domain boundaries
x_min, x_max = 0.0, 1.0  # x-axis boundaries
y_min, y_max = 0.0, 1.0  # y-axis boundaries

# Apply reflective boundary conditions
for i in range(N):
    # Reflect particles at x-axis boundaries
    if r[i, 0] < x_min:
        r[i, 0] = 2 * x_min - r[i, 0]
        v[i, 0] = -v[i, 0]
    elif r[i, 0] > x_max:
        r[i, 0] = 2 * x_max - r[i, 0]
        v[i, 0] = -v[i, 0]

    # Reflect particles at y-axis boundaries
    if r[i, 1] < y_min:
        r[i, 1] = 2 * y_min - r[i, 1]
        v[i, 1] = -v[i, 1]
    elif r[i, 1] > y_max:
        r[i, 1] = 2 * y_max - r[i, 1]
        v[i, 1] = -v[i, 1]

# Print the final positions and velocities after applying boundary conditions
print("Final Positions:", r)
print("Final Velocities:", v)

In [ ]:

import numpy as np
from scipy.spatial.distance import cdist

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
h = 0.1  # Smoothing length

# Perform neighbor search
neighbors = []
for i in range(N):
    distances = cdist(r[i].reshape(1, 2), r)  # Compute pairwise distances between particles
    indices = np.where(distances <= h)[1]  # Find indices of particles within smoothing length
    neighbors.append(indices.tolist())

# Print the neighbors for each particle
for i, particle_neighbors in enumerate(neighbors):
    print("Particle", i, "Neighbors:", particle_neighbors)

In [ ]:

import numpy as np

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Simulation domain boundaries
x_min, x_max = 0.0, 1.0  # x-axis boundaries
y_min, y_max = 0.0, 1.0  # y-axis boundaries

# Apply boundary handling using ghost particles
for i in range(N):
    # Reflect particles at x-axis boundaries
    if r[i, 0] < x_min:
        r[i, 0] = 2 * x_min - r[i, 0]
        v[i, 0] = -v[i, 0]
    elif r[i, 0] > x_max:
        r[i, 0] = 2 * x_max - r[i, 0]
        v[i, 0] = -v[i, 0]

    # Reflect particles at y-axis boundaries
    if r[i, 1] < y_min:
        r[i, 1] = 2 * y_min - r[i, 1]
        v[i, 1] = -v[i, 1]
    elif r[i, 1] > y_max:
        r[i, 1] = 2 * y_max - r[i, 1]
        v[i, 1] = -v[i, 1]

    # Create ghost particles outside the domain
    if r[i, 0] < x_min or r[i, 0] > x_max or r[i, 1] < y_min or r[i, 1] > y_max:
        ghost_particle = np.copy(r[i])
        ghost_particle_velocity = np.copy(v[i])

        # Apply appropriate forces to the ghost particle
        # ...

# Print the final positions and velocities after applying boundary handling
print("Final Positions:", r)
print("Final Velocities:", v)

In [ ]:

import numpy as np

# Particle properties
N = 100  # Number of particles
r = np.random.uniform(low=0.0, high=1.0, size=(N, 2))  # 2D positions of particles (randomly initialized)
v = np.random.uniform(low=-1.0, high=1.0, size=(N, 2))  # 2D velocities of particles (randomly initialized)

# Simulation domain boundaries
x_min, x_max = 0.0, 1.0  # x-axis boundaries
y_min, y_max = 0.0, 1.0  # y-axis boundaries

# Apply boundary handling using ghost particles
for i in range(N):
    # Reflect particles at x-axis boundaries
    if r[i, 0] < x_min:
        r[i, 0] = 2 * x_min - r[i, 0]
        v[i, 0] = -v[i, 0]
    elif r[i, 0] > x_max:
        r[i, 0] = 2 * x_max - r[i, 0]
        v[i, 0] = -v[i, 0]

    # Reflect particles at y-axis boundaries
    if r[i, 1] < y_min:
        r[i, 1] = 2 * y_min - r[i, 1]
        v[i, 1] = -v[i, 1]
    elif r[i, 1] > y_max:
        r[i, 1] = 2 * y_max - r[i, 1]
        v[i, 1] = -v[i, 1]

    # Create ghost particles outside the domain
    if r[i, 0] < x_min or r[i, 0] > x_max or r[i, 1] < y_min or r[i, 1] > y_max:
        ghost_particle = np.copy(r[i])
        ghost_particle_velocity = np.copy(v[i])

        # Apply appropriate forces to the ghost particle
        # ...

# Print the final positions and velocities after applying boundary handling
print("Final Positions:", r)
print("Final Velocities:", v)

In [ ]:

import numpy as np
from sklearn.neighbors import KDTree

# Particle properties
N = 100  # Number of particles
mass = np.ones(N)  # Mass of particles
pos = np.random.uniform(low=-1.0, high=1.0, size=(N, 3))  # 3D positions of particles (randomly initialized)

# Constants
G = 6.67430e-11  # Gravitational constant

# Build KD-Tree for efficient neighbor search
tree = KDTree(pos)

# Compute gravitational forces
forces = np.zeros_like(pos)
for i in range(N):
    # Find neighbors within a certain distance (e.g., using a threshold or a desired accuracy)
    neighbors = tree.query_radius([pos[i]], r=0.5)[0]  # Example: search radius of 0.5

    for j in neighbors:
        if i != j:
            r = pos[j] - pos[i]  # Distance vector between particles
            dist = np.linalg.norm(r)  # Magnitude of the distance vector
            force = G * mass[i] * mass[j] / dist**3 * r  # Gravitational force
            forces[i] += force

# Print the gravitational forces for each particle
for i, force in enumerate(forces):
    print("Particle", i, "Gravitational Force:", force)

In [ ]:

import numpy as np
import scipy.fftpack as fftpack

# Particle properties
N = 100  # Number of particles
mass = np.ones(N)  # Mass of particles
pos = np.random.uniform(low=-1.0, high=1.0, size=(N, 3))  # 3D positions of particles (randomly initialized)

# Constants
G = 6.67430e-11  # Gravitational constant

# Grid properties
M = 64  # Number of grid points per dimension
L = 2.0  # Length of the simulation domain
dx = L / M  # Grid spacing

# Compute the density field on the grid
density = np.zeros((M, M, M))
for p in pos:
    indices = np.floor((p + L / 2) / dx).astype(int)
    density[indices[0], indices[1], indices[2]] += 1

# Perform Fourier transform of the density field
density_fft = fftpack.fftn(density)

# Compute the gravitational potential in Fourier space
kx = np.fft.fftfreq(M, dx)
kx, ky, kz = np.meshgrid(kx, kx, kx, indexing='ij')
k_squared = kx**2 + ky**2 + kz**2
k_squared[0, 0, 0] = 1.0  # Avoid division by zero
potential_fft = -G * fftpack.ifftn(density_fft / k_squared)

# Perform inverse Fourier transform to obtain the gravitational potential in real space
potential = fftpack.ifftn(potential_fft)

# Compute the gravitational forces from the potential
forces = np.zeros_like(pos)
for i, p in enumerate(pos):
    indices = np.floor((p + L / 2) / dx).astype(int)
    force_x = -(potential[indices[0] + 1, indices[1], indices[2]] - potential[indices[0] - 1, indices[1], indices[2]]) / (2 * dx)
    force_y = -(potential[indices[0], indices[1] + 1, indices[2]] - potential[indices[0], indices[1] - 1, indices[2]]) / (2 * dx)
    force_z = -(potential[indices[0], indices[1], indices[2] + 1] - potential[indices[0], indices[1], indices[2] - 1]) / (2 * dx)
    forces[i] = np.array([force_x, force_y, force_z]) * mass[i]

# Print the gravitational forces for each particle
for i, force in enumerate(forces):
    print("Particle", i, "Gravitational Force:", force)

In [ ]:

import numpy as np
import scipy.fftpack as fftpack

# Particle properties
N = 100  # Number of particles
mass = np.ones(N)  # Mass of particles
pos = np.random.uniform(low=-1.0, high=1.0, size=(N, 3))  # 3D positions of particles (randomly initialized)

# Constants
G = 6.67430e-11  # Gravitational constant

# Grid properties
M = 64  # Number of grid points per dimension
L = 2.0  # Length of the simulation domain
dx = L / M  # Grid spacing

# Compute the density field on the grid
density = np.zeros((M, M, M))
for p in pos:
    indices = np.floor((p + L / 2) / dx).astype(int)
    density[indices[0], indices[1], indices[2]] += 1

# Perform Fourier transform of the density field
density_fft = fftpack.fftn(density)

# Compute the gravitational potential in Fourier space
kx = np.fft.fftfreq(M, dx)
kx, ky, kz = np.meshgrid(kx, kx, kx, indexing='ij')
k_squared = kx**2 + ky**2 + kz**2
k_squared[0, 0, 0] = 1.0  # Avoid division by zero
potential_fft = -G * fftpack.ifftn(density_fft / k_squared)

# Perform inverse Fourier transform to obtain the gravitational potential in real space
potential = fftpack.ifftn(potential_fft)

# Compute the gravitational forces from the potential
forces = np.zeros_like(pos)
for i, p in enumerate(pos):
    indices = np.floor((p + L / 2) / dx).astype(int)
    force_x = -(potential[indices[0] + 1, indices[1], indices[2]] - potential[indices[0] - 1, indices[1], indices[2]]) / (2 * dx)
    force_y = -(potential[indices[0], indices[1] + 1, indices[2]] - potential[indices[0], indices[1] - 1, indices[2]]) / (2 * dx)
    force_z = -(potential[indices[0], indices[1], indices[2] + 1] - potential[indices[0], indices[1], indices[2] - 1]) / (2 * dx)
    forces[i] = np.array([force_x, force_y, force_z]) * mass[i]

# Print the gravitational forces for each particle
for i, force in enumerate(forces):
    print("Particle", i, "Gravitational Force:", force)

In [ ]:

# Initialize density array
density = np.zeros(N)

# Loop over particles
for i in range(N):
    # Reset density to initial value
    density[i] = initial_density

    # Loop over neighboring particles
    for j in neighbors[i]:
        # Calculate distance between particles i and j
        r_ij = np.linalg.norm(position[i] - position[j])

        # Compute the contribution to density using the smoothing kernel function
        density[i] += mass[j] * smoothing_kernel(r_ij, h)

In [ ]:

import numpy as np

# Assuming you have position arrays for particles i and j
position_i = np.array([...])  # Shape: (N,)
position_j = np.array([...])  # Shape: (N,)

# Calculate the relative position vector r_ij
r_ij = position_i - position_j  # Shape: (N,)

# Assuming you have known gravitational forces F(r_ij) from the SPH simulation
known_forces = np.array([...])  # Shape: (N,)

# Create input and output variables for training the neural network
input_data = r_ij  # Shape: (N,)
output_data = known_forces  # Shape: (N,)

In [ ]:

import tensorflow as tf
from tensorflow import keras

# Define the neural network architecture
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(output_dim)  # output_dim corresponds to the force vector dimensions
])

# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=num_epochs, batch_size=batch_size)

In [ ]:

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Particle Positions
fig = plt.figure(figsize=(10, 5))

# 2D Plot
ax1 = fig.add_subplot(121)
ax1.scatter(position[:, 0], position[:, 1], s=5, c='blue')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_title('Particle Positions (2D)')

# 3D Plot
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(position[:, 0], position[:, 1], position[:, 2], s=5, c='blue')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.set_title('Particle Positions (3D)')

plt.tight_layout()
plt.show()

In [ ]:

plt.contour(x, y, density)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Density Contour Plot')
plt.colorbar()
plt.show()

In [ ]:

plt.quiver(x, y, velocity_x, velocity_y)
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Velocity Field (2D)')
plt.show()

In [ ]:

plt.imshow(temperature, extent=[x_min, x_max, y_min, y_max], origin='lower')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Temperature Map')
plt.colorbar()
plt.show()